# DSA210 Term Project — Part 3: Machine Learning (GPR → Bitcoin & Gold)

This notebook trains multiple regression models to predict **Bitcoin** and **Gold** returns using **Geopolitical Risk (GPR)** indicators.

## What you need
- A processed dataset CSV produced in Part 2 (or earlier) named: `processed_gpr_btc_gold.csv`
- Columns expected:
  - Features: `GPR, GPRH, GPRT, GPR_LAG1, GPR_LAG3, GPR_ROLL3`
  - Targets: `btc_return, gold_return`

> If your column names differ, edit the `features` list and target column names in the **Data Setup** cell.


In [ ]:
# =========================
# 1) Imports
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

sns.set_context("talk")


In [ ]:
# =========================
# 2) PART B — Load & Clean 3 Datasets (GPR + Bitcoin + Gold)
# =========================
import pandas as pd
import numpy as np

# --- (1) Helper: robust CSV/XLS loading (local first, then GitHub raw) ---
def read_any(path_local, github_raw_url=None, **kwargs):
    """Try reading from local path first; if it fails and github_raw_url is provided, read from GitHub raw."""
    try:
        if path_local.lower().endswith(('.xls', '.xlsx')):
            return pd.read_excel(path_local, **kwargs)
        return pd.read_csv(path_local, **kwargs)
    except Exception as e:
        if github_raw_url is None:
            raise
        print(f"Local read failed for {path_local} -> trying GitHub raw...")
        if github_raw_url.lower().endswith(('.xls', '.xlsx')):
            return pd.read_excel(github_raw_url, **kwargs)
        return pd.read_csv(github_raw_url, **kwargs)

# ===========================================================
# A) GPR (Geopolitical Risk) — Excel (.xls)
# ===========================================================
GPR_PATH_LOCAL = "data_gpr_export.xls"   # put next to the notebook OR use your repo path like: data/data_gpr_export.xls
GPR_PATH_GITHUB = None  # optional: raw GitHub link if you want

df_gpr_raw = read_any(GPR_PATH_LOCAL, GPR_PATH_GITHUB)

# --- Clean GPR ---
df_gpr = df_gpr_raw.copy()

# 1) Identify date column
possible_date_cols = [c for c in df_gpr.columns if str(c).lower() in ["date", "dates", "time", "month", "timestamp"]]
if possible_date_cols:
    date_col = possible_date_cols[0]
else:
    # if the first column looks like a date, use it
    date_col = df_gpr.columns[0]

df_gpr[date_col] = pd.to_datetime(df_gpr[date_col], errors="coerce")
df_gpr = df_gpr.dropna(subset=[date_col]).sort_values(date_col)
df_gpr = df_gpr.rename(columns={date_col: "Date"})

# 2) Standardize expected GPR columns (adapt if your file uses different names)
rename_map = {}
for c in df_gpr.columns:
    lc = str(c).strip().lower()
    if lc in ["gpr", "gpr index", "geopolitical risk"]: rename_map[c] = "GPR"
    if lc in ["gprh", "gpr_h", "gpr high", "gpr headline"]: rename_map[c] = "GPRH"
    if lc in ["gprt", "gpr_t", "gpr threats"]: rename_map[c] = "GPRT"
df_gpr = df_gpr.rename(columns=rename_map)

# Keep only Date + available GPR columns
keep_gpr_cols = [c for c in ["Date", "GPR", "GPRH", "GPRT"] if c in df_gpr.columns]
df_gpr = df_gpr[keep_gpr_cols].copy()

# Convert to numeric (some excel exports have commas/strings)
for c in ["GPR", "GPRH", "GPRT"]:
    if c in df_gpr.columns:
        df_gpr[c] = pd.to_numeric(df_gpr[c], errors="coerce")

# Remove duplicates and set index
df_gpr = df_gpr.drop_duplicates(subset=["Date"]).set_index("Date").sort_index()

# If GPR is monthly, align to daily by forward-filling to business days
# (we'll ultimately merge with daily BTC/Gold)
df_gpr = df_gpr.asfreq('B').ffill()

print("GPR cleaned:", df_gpr.shape, "columns:", list(df_gpr.columns))

# ===========================================================
# B) Bitcoin Prices — CSV
# ===========================================================
# Use your own file names; the code tries to be flexible.
BTC_PATH_LOCAL = "btc.csv"   # e.g., BTC-USD.csv (Yahoo) or your own export
BTC_PATH_GITHUB = None

df_btc_raw = read_any(BTC_PATH_LOCAL, BTC_PATH_GITHUB)

df_btc = df_btc_raw.copy()

# Standardize Date column
btc_date_candidates = [c for c in df_btc.columns if str(c).lower() in ["date", "time", "timestamp"]]
btc_date_col = btc_date_candidates[0] if btc_date_candidates else df_btc.columns[0]
df_btc[btc_date_col] = pd.to_datetime(df_btc[btc_date_col], errors="coerce")
df_btc = df_btc.dropna(subset=[btc_date_col]).sort_values(btc_date_col)
df_btc = df_btc.rename(columns={btc_date_col: "Date"}).set_index("Date")

# Choose price column (Adj Close preferred)
price_candidates = [c for c in df_btc.columns if str(c).lower() in ["adj close", "adj_close", "close", "price"]]
btc_price_col = price_candidates[0] if price_candidates else df_btc.columns[-1]
df_btc[btc_price_col] = pd.to_numeric(df_btc[btc_price_col], errors="coerce")
df_btc = df_btc[[btc_price_col]].rename(columns={btc_price_col: "BTC_Price"}).dropna()

# Daily log returns (more stable)
df_btc["btc_return"] = np.log(df_btc["BTC_Price"]).diff()
df_btc = df_btc.asfreq('B').ffill()

print("BTC cleaned:", df_btc.shape)

# ===========================================================
# C) Gold Prices — CSV
# ===========================================================
GOLD_PATH_LOCAL = "gold.csv"  # e.g., GC=F.csv (Yahoo) or LBMA series export
GOLD_PATH_GITHUB = None

df_gold_raw = read_any(GOLD_PATH_LOCAL, GOLD_PATH_GITHUB)

df_gold = df_gold_raw.copy()

gold_date_candidates = [c for c in df_gold.columns if str(c).lower() in ["date", "time", "timestamp"]]
gold_date_col = gold_date_candidates[0] if gold_date_candidates else df_gold.columns[0]
df_gold[gold_date_col] = pd.to_datetime(df_gold[gold_date_col], errors="coerce")
df_gold = df_gold.dropna(subset=[gold_date_col]).sort_values(gold_date_col)
df_gold = df_gold.rename(columns={gold_date_col: "Date"}).set_index("Date")

gold_price_candidates = [c for c in df_gold.columns if str(c).lower() in ["adj close", "adj_close", "close", "price"]]
gold_price_col = gold_price_candidates[0] if gold_price_candidates else df_gold.columns[-1]
df_gold[gold_price_col] = pd.to_numeric(df_gold[gold_price_col], errors="coerce")
df_gold = df_gold[[gold_price_col]].rename(columns={gold_price_col: "Gold_Price"}).dropna()

df_gold["gold_return"] = np.log(df_gold["Gold_Price"]).diff()
df_gold = df_gold.asfreq('B').ffill()

print("Gold cleaned:", df_gold.shape)

# ===========================================================
# D) Merge & Feature Engineering (create ONE modeling table)
# ===========================================================
df = df_gpr.join(df_btc[["btc_return"]], how="inner").join(df_gold[["gold_return"]], how="inner")

# Optional: drop first NA due to diff()
df = df.dropna()

# --- GPR features (lags + rolling mean) ---
# Use whichever GPR columns exist
base_gpr_col = "GPR" if "GPR" in df.columns else df.columns[0]

df["GPR_LAG1"] = df[base_gpr_col].shift(1)
df["GPR_LAG3"] = df[base_gpr_col].shift(3)
df["GPR_ROLL3"] = df[base_gpr_col].rolling(3).mean()

df = df.dropna()

print("Final merged dataset:", df.shape)
df.head()


In [ ]:
# =========================
# 2.5) Modeling Table Setup (Features + Targets)
# =========================
# Targets: daily log returns
y_btc = df['btc_return']
y_gold = df['gold_return']

# Features: all columns except targets
X = df.drop(columns=['btc_return','gold_return'])

print('Feature columns:', list(X.columns))
print('X shape:', X.shape)


In [ ]:
# =========================
# 3) Time-aware Train/Test split
# =========================
split = int(len(df) * 0.7)

X_train, X_test = X.iloc[:split], X.iloc[split:]
y_btc_train, y_btc_test = y_btc.iloc[:split], y_btc.iloc[split:]
y_gold_train, y_gold_test = y_gold.iloc[:split], y_gold.iloc[split:]

print("Train size:", len(X_train), "| Test size:", len(X_test))


In [ ]:
# =========================
# 4) Standardization
# =========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Models
We train three models:
- Linear Regression (baseline)
- Ridge Regression (regularized linear)
- Random Forest Regressor (non-linear)

In [ ]:
# =========================
# 5) Model definitions
# =========================
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Random Forest": RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )
}


In [ ]:
# =========================
# Helper: Train & evaluate
# =========================
def train_and_evaluate(target_name, y_train, y_test):
    rows = []
    trained = {}
    preds = {}
    for name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
        r2 = float(r2_score(y_test, y_pred))

        rows.append({"Asset": target_name, "Model": name, "RMSE": rmse, "R2": r2})
        trained[name] = model
        preds[name] = y_pred
    return pd.DataFrame(rows), trained, preds

results_btc, trained_btc, preds_btc = train_and_evaluate("Bitcoin", y_btc_train, y_btc_test)
results_gold, trained_gold, preds_gold = train_and_evaluate("Gold", y_gold_train, y_gold_test)

results = pd.concat([results_btc, results_gold], ignore_index=True)
results


In [ ]:
# =========================
# 6) Performance comparison (R2)
# =========================
plt.figure(figsize=(10, 5))
sns.barplot(data=results, x="R2", y="Model", hue="Asset")
plt.title("Model Performance Comparison (R2)")
plt.xlim(-0.2, 1.0)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
sns.barplot(data=results, x="RMSE", y="Model", hue="Asset")
plt.title("Model Performance Comparison (RMSE)")
plt.tight_layout()
plt.show()


In [ ]:
# =========================
# 7) Diagnostic plots (Actual vs Predicted + Residuals)
# =========================
def plot_performance(model_name, y_true, y_pred, asset_name):
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Actual vs Pred
    sns.scatterplot(x=y_true, y=y_pred, alpha=0.6, ax=axes[0])
    min_val, max_val = min(float(y_true.min()), float(np.min(y_pred))), max(float(y_true.max()), float(np.max(y_pred)))
    axes[0].plot([min_val, max_val], [min_val, max_val], linestyle="--", linewidth=2)
    axes[0].set_title(f"{asset_name} — {model_name}: Actual vs Predicted")
    axes[0].set_xlabel("Actual Return")
    axes[0].set_ylabel("Predicted Return")

    # Residuals
    residuals = y_true - y_pred
    sns.scatterplot(x=y_pred, y=residuals, alpha=0.6, ax=axes[1])
    axes[1].axhline(0, linestyle="--", linewidth=2)
    axes[1].set_title(f"{asset_name} — {model_name}: Residual Plot")
    axes[1].set_xlabel("Predicted Return")
    axes[1].set_ylabel("Residuals (Actual - Predicted)")

    plt.tight_layout()
    plt.show()

# Bitcoin plots
for name, y_pred in preds_btc.items():
    plot_performance(name, y_btc_test.reset_index(drop=True), pd.Series(y_pred), "Bitcoin")

# Gold plots
for name, y_pred in preds_gold.items():
    plot_performance(name, y_gold_test.reset_index(drop=True), pd.Series(y_pred), "Gold")


In [ ]:
# =========================
# 8) Feature importance (Random Forest)
# =========================
def plot_rf_importance(trained_model, title):
    importances = pd.Series(trained_model.feature_importances_, index=features).sort_values()
    plt.figure(figsize=(10, 6))
    importances.plot(kind="barh")
    plt.title(title)
    plt.xlabel("Importance")
    plt.tight_layout()
    plt.show()

plot_rf_importance(trained_btc["Random Forest"], "Bitcoin — Random Forest Feature Importance")
plot_rf_importance(trained_gold["Random Forest"], "Gold — Random Forest Feature Importance")


## Simple scenario analysis (High GPR)
This function predicts returns under a hypothetical high-risk environment by setting GPR/GPRH/GPRT to chosen values and mirroring them into lag/rolling features.

In [ ]:
# =========================
# 9) Scenario prediction helper
# =========================
def predict_high_gpr_scenario(gpr, gprh, gprt, asset="Bitcoin"):
    input_df = pd.DataFrame([{
        "GPR": gpr,
        "GPRH": gprh,
        "GPRT": gprt,
        "GPR_LAG1": gpr,
        "GPR_LAG3": gpr,
        "GPR_ROLL3": gpr
    }])[features]

    input_scaled = scaler.transform(input_df)

    if asset.lower() == "bitcoin":
        trained = trained_btc
        print("\n--- HIGH GPR SCENARIO (Bitcoin) ---")
    else:
        trained = trained_gold
        print("\n--- HIGH GPR SCENARIO (Gold) ---")

    print(f"GPR={gpr}, GPRH={gprh}, GPRT={gprt}")
    for name, model in trained.items():
        pred = float(model.predict(input_scaled)[0])
        print(f"{name} prediction: {pred:.6f}")

# Example: adjust values based on your dataset scale (e.g., try df['GPR'].quantile(0.9))
predict_high_gpr_scenario(gpr=float(df["GPR"].quantile(0.9)),
                          gprh=float(df["GPRH"].quantile(0.9)),
                          gprt=float(df["GPRT"].quantile(0.9)),
                          asset="Bitcoin")
predict_high_gpr_scenario(gpr=float(df["GPR"].quantile(0.9)),
                          gprh=float(df["GPRH"].quantile(0.9)),
                          gprt=float(df["GPRT"].quantile(0.9)),
                          asset="Gold")


## Export results table for your report
This saves `part3_model_results.csv` which you can reference in your write-up.

In [ ]:
results.to_csv("part3_model_results.csv", index=False)
print("Saved: part3_model_results.csv")